# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación\*

### Minería de Datos: Curso académico 2020-2021

### Compañeros:Juan Ibañez y Raul Milla

# 1.Preliminares

Iniciamos las distintas bibliotecas y script necesarios para 

In [ ]:
# Third party
from matplotlib import pyplot as mpl
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, Normalizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pandas as panda
from imblearn.over_sampling import SMOTE
# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils
seed = 27912

## 2.Analizamos la base de datos


Esta base de datos nos cuenta basicamente si una persona tiene un cancer benigno o maligno(este atributo sera el objetivo de nuestro sistema)por otro lado tenemos otros atributos que nos indican la media,el error y la desviacion de algunos datos sobre las celulas del paciente.
Lo primero que haremos sera dividir los diferentes **casos** entre test y training,mas tarde estos casos se dividiran por sus **variables**(separaremos el objetivo de los demas).Antes de esto tendriamos que eliminar una **variable**  en todos los casos,la ultima que es un null y no aporta nada al estudio.Luego tendriamos que buscar aquellas **variables** que tampoco aportan mucho al estudio para ello buscaremos su correlacion con las demas ,pero antes lo normalizaremso dividiendo cada **variable** entre el valor maximo de todos los casos


Agregamos los datos

In [ ]:
filepathC = "../input/breast-cancer-wisconsin-data/data.csv"

indexC = "id"
targetC = "diagnosis"


dataC = utils.load_data(filepathC, indexC, targetC)

Una vez cargado hacemos una prueba para comprobar que efectivamente se ha cargado la BBDD,haciendo una tirada de las 5 primeras

In [ ]:
dataC.head(5)

Vemos que añade una tabla que sobra,por lo que eliminamos

In [ ]:
dataC = dataC.drop(dataC.columns[31], axis='columns')


Volvemos a comprobar que esta vez la tabla este bien y que ya no aparece la ultima columna

In [ ]:
dataC.head(5)

Determinados que son los atributos y cuales los target

In [ ]:
(X, y) = utils.divide_dataset(dataC, target=targetC)

Comprobamos

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Una vez que comprobamos que se realiza correctamente determinamos que cantidad de datos usamos para el entrenamiento(un 70%).Tambien definimos las X e Y de test

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Hacemos una comprobacion de que se ha cargado

In [ ]:
X_train.sample(5, random_state=seed)


In [ ]:
X_test.sample(5, random_state=seed)


In [ ]:
y_train.sample(5, random_state=seed)


In [ ]:
y_test.sample(5, random_state=seed)

Procedemos a juntar los atributos con los target

In [ ]:
dataC_train = utils.join_dataset(X_train, y_train)
dataC_test = utils.join_dataset(X_test, y_test)

# 1.Analisis de los datos de CW

Comprobamos cuantos **casos** hay 

In [ ]:
dataC_train.shape

Hay 398 casos y 31 variables

In [ ]:
dataC_train.info(memory_usage=False)

Asi vemos que no hay ningun dato dañado,es decir que todos los casos tienes sus variables

In [ ]:
utils.plot_histogram(dataC_train)

Al haber tantas **variables** no podemos verlo claramente y no solo eso al no estar normalizado tenemos **variables** que van a lo mejor de 0 a 100 y otros que van hasta mas de 4000 por lo que para ver todos los datos juntos debemos de normalizarlo entre el maximo de cada uno para que todo vaya de 0 a 1(esto lo haremos mas adelante)
 

In [ ]:
utils.plot_barplot(dataC_train)

In [ ]:
utils.plot_pairplot(dataC_train, target=targetC)

Como podemos ver no podemos ver casi nada,por lo que tenemos que restringir los datos.
Tambien ya podemos eliminar algunos datos como por ejemplo perimetro y area ya que son dependientes del radio,pero en vez de eso haremos una tabla de coorelacion


Ahora para poder constrastar datos debemos de separar diagnosis de los demas

In [ ]:
dataC_trainC,dataC_trainO=(dataC_train.drop("diagnosis",axis=1),dataC_train["diagnosis"])

Para buscar que otro atributos no nos son utiles,dividimos cada columna por su maximo

In [ ]:
columnas=[]
for i in dataC.drop(['diagnosis'],axis='columns'):
    maximo=max(dataC[i])
    dataC[i]=dataC[i]/maximo
    columnas.append(i)
for i in dataC_train.drop(['diagnosis'],axis='columns'):
    maximo=max(dataC_train[i])
    dataC_train[i]=dataC_train[i]/maximo


Ahora tenemos que observar los datos:

In [ ]:
f,ax=mpl.subplots(figsize=(31,31))
corrMatrix = dataC_trainC.corr()

sns.heatmap(corrMatrix,annot=True,linewidths=.5,fmt=".1f",ax=ax)


A partir de esto podemos ver que hay algunos datos que por asi decirlo sobran ya que tienen mucha correlacion,asi que lo que haremos sera una funcion para que localice aquellos que tienen una correlacion de 0.9 o mas y luego eliminarlo

In [ ]:
categorias=dataC_trainC.columns
x=0
aux=x
eliminado=[]
for i in categorias:
    aux=x+1
    no_encontrado=True
    while (aux<len(categorias) and no_encontrado and not(i in eliminado)  ):
        #print(eliminado)
        if(corrMatrix[i][categorias[aux]]>0.8):
            #print([corrMatrix[i][categorias[aux]]])
            no_encontrado=False
            eliminado.append(i)
            
        aux=aux+1
    x=x+1
for i in eliminado:
    dataC_trainC=dataC_trainC.drop(i,axis=1)
    dataC_train=dataC_train.drop(i,axis=1)
print(eliminado)

In [ ]:
f,ax=mpl.subplots(figsize=(len(dataC_trainC.columns),len(dataC_trainC.columns)))
corrMatrix = dataC_trainC.corr()

sns.heatmap(corrMatrix,annot=True,linewidths=.5,fmt=".1f",ax=ax)



Ahora podemos ver que no hay ninguna correlacion mayor que 0.9

In [ ]:
utils.plot_barplot(dataC_train)

In [ ]:
utils.plot_pairplot(dataC_train, target="diagnosis")

In [ ]:
utils.plot_histogram(dataC_trainC)

Por ultimo debemos de separar los datos del test como ya hicimos anteriormente

In [ ]:
dataC_testC,dataC_testO=(dataC_test.drop("diagnosis",axis=1),dataC_test["diagnosis"])

# 4.Prepocesamiento de datos

Ahora tenemos que hacer varias tareas:
* Limpieza de datos
* Integracion de datos
* Transformacion de datos
* Reduccion de datos
Las dos ultimas ya se han realizado con anterioridad,y sobre la limpieza ya hemos visto que no es necesario ya que no hay datos anomalos o nuloss 

# Discretizacion

In [ ]:
best_features = SelectKBest(chi2, 4)
l1_normalizer = Normalizer("l1")


# 5.Algoritmos de classificacion y evaluacion

Empezamos Algoritmo Zero-R

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")
utils.evaluate(zero_r_model,
               dataC_trainC, dataC_testC,
               dataC_trainO, dataC_testO)

Como era de esperar no nos da mucha confianza ya que basicamente coge el valor que mas se repite en el train y se lo aplica a todos los test

Ahora probaremos con el modelo tree_model

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)


Para este caso debemos de reducir el test ya que tiene mas atributos que el train

In [ ]:
dataC_testC_red=dataC_testC
for i in eliminado:
    dataC_testC_red=dataC_testC_red.drop(i,axis=1)

In [ ]:
utils.evaluate(tree_model,
               dataC_trainC, dataC_testC_red,
               dataC_trainO, dataC_testO)

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="uniform")
discretize_tree_model = make_pipeline(discretizer, tree_model)
utils.evaluate(discretize_tree_model,
               dataC_trainC, dataC_testC_red,
               dataC_trainO, dataC_testO)

Tras hacer las diferentes pruebas vemos que el mejor algoritmo que tenemos es el modelo de arbol pero sin descretizar con un error de menos de un 8 por cierto y vemos que el peor obviamente ha sido el de Zero ya que ese nos da un error muy alto(casi el 40%).Por otro lado no podria decir cual es mejor y cual se recomieda usar por que por un lado el algortimo de arbo descretizado solo da un benigno mal asi que dependiendo de que es mas valioso si o bien que se equivoque en falsos positivos o en positivos falsos o en los dos.

# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación\*

### Minería de Datos: Curso académico 2020-2021

### Pima diabetes

# 1. Preliminares

In [ ]:
seed = 27912

# 2. Acceso y almacenamiento de datos

El conjunto de datos que vamos a emplear es Pima diabetes. Tiene dos posibles resultados.

0 Cuando el paciente no tiene diabetes
1 Cuando el paciente tiene diabetes
Que conforman los valores de la variable a predecir, la variable objetivo (Outcome). Se tienene en cuenta las siguientes variables predictoras:

Age: Edad del paciente.
Pregnancies: Número de embarazos de la paciente.
Glucose: Concentración de glucosa en plasma sanguíneo.
BloodPressure: Presión diastólica arterial (mm Hg).
SkinThickness: Grosor de la piel en el triceps (mm).
insulin: Cantidad de insulina en sangre (mu Insulina/ml).
BMI: Índice de Masa Corporal (kg/m^2).
DiabetesPedigreeFunction: Función para el historial de diabetes en la familia del paciente.

Cargamos el data-set

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = None
target = "Outcome"

data = utils.load_data(filepath, index, target)

En el caso de Pima Diabetes no disponemos de variable índice, asi que index lo ponemos a "none"
 Usamos la función head para obtener las n primeras instancias del conjunto de datos:

In [ ]:
data.head(5)

Pero para conseguir una muestra menos sesgada hacemos una aleatoria, porque los data-set suelen estar ordenanos en función de sus variables.

In [ ]:
data.sample(5, random_state=seed)

Creamos conjunto de datos separado dos subconjuntos, uno con las variables predictoras (X) y otro con la variable objetivo (y)

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Comprobamos que los conjuntos de las variables predictoras y el de la variable objetivo se han sepeparado correctamente

X.sample(5, random_state=seed)

Comprobamos también con la variable objetivo:


In [ ]:
y.sample(5, random_state=seed)

Para hacer un proceso de Holdout creamos la muestra de Test y la muestra de training con la siguiente proporción:

* Muestra training (70%)
* Muestra test (típicamente, 30%)


Para realizar un *holdout* podemos utilizar el método `train_test_split` de `scikit-learn`:


In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Comenzamos con las variables predictoras del conjunto de datos de training:

In [ ]:
X_train.sample(5, random_state=seed)

Y test:

In [ ]:
X_test.sample(5, random_state=seed)

Hacemos la muestra con el conjunto de training

In [ ]:
y_train.sample(5, random_state=seed)

Y test:

In [ ]:
y_test.sample(5, random_state=seed)

# 3. Análisis exploratorio de datos

Ahora para ver las propiedades de nuestro conjunto de datos y poder sacar conclusiones de cara al preprocesamiento usaremos gráficos y estadisticos.

### Descripción del conjunto de datos

Con shape podemos saber la cantidad de casos que hay en nuestra base de datos frente a la cantidad que variables que disponemos, 768 pacientes estudiados frente a las nueve variables que disponemos, que son las diferentes pruebas que se han hecho a estos pacientes.

Usamos el metodo info para saber de que tipo son las variables

data.info(memory_usage=False)

En nuestro caso todas las variables predictoras son varibales continuas, y solo la variable obejetivo es una variable discreta,

In [ ]:
y.cat.categories

Nuestra variable objetivo tiene dos posibles valores, 0 y 1

In [ ]:
data.shape

* ### Visualización de las variables

Vamos a comenzar visualizando las variables numéricas del conjunto de datos usando histogramas para las variables numéricas y diagramas de barras para las variables categóricas

In [ ]:
utils.plot_histogram(data)

Como podemos observar varias variables tiene tienen datos ruidosos, todos los que tienen una cantidad 0, alejados de la mayoría de valores de esas variables, este sería el caso de`Glucose` , `BloodPressure`, `SkinThickness`,`insulin` y `BMI`. Tambein podemos ver que `SkinThickness`, `Age` y `Insulin` tienen una disposición central. No encuentro valores anomalos en este analisis multivariado.

In [ ]:
utils.plot_barplot(data)

En el caso de nuestra única variable categórica podemos ver que la muestra no esta balanceada, de nuestros 768 pacientes la mayoría dio negativo en tener diabetes como nos dice este analisis univariado.

A continuacion creamos dos matrices para analizar la correlación entre pares de variables, en el primer caso el numero de cada hueco representa la coincidencia o solopamiento entre variables , que en caso de ser demasiado alta nos podria indicar la redundancia de ciertas variables predictoras

In [ ]:
f,ax=mpl.subplots(figsize=(31,31))
corrMatrix = X_train.corr()

sns.heatmap(corrMatrix,annot=True,linewidths=.5,fmt=".1f",ax=ax)

Viendo el resultado ninguna de las variables predictoras es redundante

In [ ]:
utils.plot_pairplot(data, target="Outcome")

En esta caso a diferencia de la de iris no podemos averiguar claramente el poder discriminatorio de cada variable predictora pues hay muchos datos ruidosos.

In [ ]:
data.describe(include="number")

In [ ]:
data.describe(include="category")

# 4. Preprocesamiento de datos

Lo primero que haremos para tratar los datos crudos sera una imputación de valores perdidos,hay varias variables , 'Glucose','BloodPressure','Insulin','BMI','SkinThickness', que por el contexto no deberían tener valores a 0, así que lo que haremos será sustituir esos valores por la mediana de cada uno , separando también segun la variable objetivo.

Calculamos la mediana de esas variables para usarlas a modo de marcador

In [ ]:
def medina(variable):
    sumt1=[]
    sumt2=[]
    n1=0
    n2=0
    t=0
    for i in data[variable]:
        j=data['Outcome'][t]
        t=t+1
        if (j==1 and i>0):
            n1=n1+1
            sumt1.append(i)
        elif (j ==0 and i>0):
            n2=n2+1
            sumt2.append(i)
    return (np.median(sumt1),np.median(sumt2))

In [ ]:
medina('Glucose')
medina('BloodPressure')
medina('BMI')
medina('SkinThickness')
medina('Insulin')

Con las medianas calculadas pasamos a remplazar los 0 por NaN

In [ ]:
col=['Glucose','BloodPressure','Insulin','BMI','SkinThickness']
for i in col:
    data[i].replace(0, np.nan, inplace= True)

Ahora remplazamos los NaN por las medianas calculadas

In [ ]:
data.loc[(data['Outcome'] == 0 ) & (data['Insulin'].isnull()), 'Insulin'] = 102.5
data.loc[(data['Outcome'] == 1 ) & (data['Insulin'].isnull()), 'Insulin'] = 169.5
data.loc[(data['Outcome'] == 0 ) & (data['Glucose'].isnull()), 'Glucose'] = 107
data.loc[(data['Outcome'] == 1 ) & (data['Glucose'].isnull()), 'Glucose'] = 140
data.loc[(data['Outcome'] == 0 ) & (data['SkinThickness'].isnull()), 'SkinThickness'] = 27
data.loc[(data['Outcome'] == 1 ) & (data['SkinThickness'].isnull()), 'SkinThickness'] = 32
data.loc[(data['Outcome'] == 0 ) & (data['BloodPressure'].isnull()), 'BloodPressure'] = 70
data.loc[(data['Outcome'] == 1 ) & (data['BloodPressure'].isnull()), 'BloodPressure'] = 74.5
data.loc[(data['Outcome'] == 0 ) & (data['BMI'].isnull()), 'BMI'] = 30.1
data.loc[(data['Outcome'] == 1 ) & (data['BMI'].isnull()), 'BMI'] = 34.3

In [ ]:
utils.plot_histogram(data)

Comprobamos que los valores con 0 de las variables han sido eliminados

Para tratar los datos anomalos usaremos StandarScaler que "escala" la propiedad restando por la media y diviendo por la desviación estándar, con esto conseguiremos que los datos anomalos sean menos sesgados que le resto en comparacion con el resto.

In [ ]:
yOut = data.Outcome
XOut = data.drop('Outcome', axis = 1)
columns = XOut.columns
scaler = StandardScaler()
XOut = scaler.fit_transform(XOut)
dataOut_x = panda.DataFrame(XOut, columns = columns)

Creamos nuevos subconjuntos sin los datos anomalos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataOut_x, yOut,
                                                                    stratify=yOut, random_state = seed, train_size = train_size)

Y por último discretizaremos,teniendo en cuenta lo visto en el diagrama de puntos probaremos con las tres discretizaciones , las tres con dos bins ya que nuestra variable objetivo tiene dos valores

In [ ]:
discretizerQuantile = KBinsDiscretizer(n_bins=2, strategy='quantile')

In [ ]:
discretizerUniform = KBinsDiscretizer(n_bins=2, strategy="uniform")

In [ ]:
discretizerKmeans = KBinsDiscretizer(n_bins=2, strategy="kmeans")

# 5. Algoritmos de clasificación

Genreamos modelos de ZeroR y de CART. Para ello usamos los siguientes clasificadores de la librería  scikit-learn

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

El algoritmo zero_r no nos dara mucha precisión pero nos servira como baseline, como referencia para el resto. Para que salga la clase más frecuente del subconjunto train usamos el hiperparámetro strategy="most_frequent"

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Tambien creamos una pipeline para cada una de las discretizaciones y usando el algoritmo CART

In [ ]:
discretize_tree_model_Quantile = make_pipeline(discretizerQuantile, tree_model)

In [ ]:
discretize_tree_model_Uniform = make_pipeline(discretizerUniform, tree_model)

In [ ]:
discretize_tree_model_Kmeans = make_pipeline(discretizerKmeans, tree_model)

# 5. Evaluación de modelos

* ### ZERO-R

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

El zero-r nos da la precisión más baja de los algoritmos testeados.

* ### CART sin discretización.

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

* ### CART con discretizacion de igual frecuencia (Quantile)

In [ ]:
utils.evaluate(discretize_tree_model_Quantile,
               X_train, X_test,
               y_train, y_test)

* ### CART con discretizacion de igual anchura (Uniform)

In [ ]:
utils.evaluate(discretize_tree_model_Uniform,
               X_train, X_test,
               y_train, y_test)

* ### CART con discretizacion basada en k-medias (Kmeans)

In [ ]:
utils.evaluate(discretize_tree_model_Kmeans,
               X_train, X_test,
               y_train, y_test)

omo podemos ver ninguna de las tres discretizaciones resultan en una mejora del algoritmo CART, todo lo contrario pues la precisión del algortimo es sensible superior sin ninguna discretización, aun así podemos observar que la que da mejor resultado de las tres es la discretización de igual anchura.